In [ ]:
# 0. Установка зависимостей (если ещё не установлено)
!pip install --quiet ultralytics tqdm pyyaml pillow

import cv2
from glob import glob
from tqdm import tqdm
import shutil
import re
import os
import numpy as np
import random
import yaml
import pandas as pd
from ultralytics import YOLO
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# # === 1. Параметры ===
# WORK_DIR   = "/kaggle/working"
# LOCAL_BASE = "/kaggle/input/sliced3"   # ваш новый датасет
# NUM_TRAIN  = 
# NUM_VAL    = 5000
# IMG_SIZE       = 640     # размер картинок для модели
# BATCH          = 8
# EPOCHS         = 3
# CONF_THRES     = 0.25
# IOU_THRES      = np.arange(0.3, 0.931, 0.07)


# # 0. Установка зависимостей (если ещё не установлено)
# !pip install --quiet ultralytics pyyaml

# import os
# import yaml
# from ultralytics import YOLO


# === 1. Параметры путей и обучения ===
WORK_DIR    = '/kaggle/working'
LIST_DIR    = '/kaggle/input/yolo-8m-80-20'
TRAIN_LIST    = os.path.join(LIST_DIR, 'train.txt')   # ваш уже существующий train.txt
VAL_LIST      = os.path.join(LIST_DIR, 'val.txt')     # ваш уже существующий val.txt
WEIGHTS     = '/kaggle/input/yolo8m-13epochs-train/pytorch/default/1/epoch1.pt'

IMG_SIZE    = 640
BATCH       = 16
EPOCHS      = 5

CONF_THRES   = 0.1
IOU_THRES      = np.arange(0.3, 0.931, 0.07)

# === 2. Data config как dict ===
data_cfg = {
    'path': '/',         # не используется, т.к. train/val — абсолютные пути
    'train': TRAIN_LIST,
    'val':   VAL_LIST,
    'nc':    1,
    'names': ['person']
}
data_yaml_path = os.path.join(WORK_DIR, 'data.yaml')
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_cfg, f)

# === 3. Гиперпараметры и аугментации ===
HYP = {
    # геометрические
    'degrees':     180,
    'translate':   0.08,
    'scale':       0.20,
    'shear':       0.0,
    'perspective': 0.0005,
    'fliplr':      0.5,
    'flipud':      0.5,
    # цветовые
    'hsv_h': 0.015,
    'hsv_s': 0.40,
    'hsv_v': 0.30,
    'bgr':   0.0,
    # optimizer
    'lr0':          0.003,
    'lrf':          0.1,
    'momentum':     0.937,
    'weight_decay': 0.0005,
    # warmup
    'warmup_epochs':  0.0,
    'warmup_bias_lr': 0.0,
    # составные
    'mosaic': 1.0,
    'mixup':  0.15,
}

# === 4. Тренировка ===
model = YOLO(WEIGHTS)
model.train(
    data=data_yaml_path,    # теперь строка с путём к YAML
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    device=0,
    augment=True,
    project=WORK_DIR,
    name='yolov8s-finetune',
    conf=CONF_THRES,
    save=True,         # включаем сохранение чекпоинтов
    save_period=1,     # сохранять каждый epoch
    **HYP
)




# === 3. Функции для оценки ===
def get_label_path(img_path):
    """
    По пути к картинке (img_path) находит правильную папку с разметкой:
    - sliced1:   .../slices/...  → .../labels/...
    - sliced2:   .../images1/... → .../labels1/...   (и аналогично images2 → labels2)
    - sliced3:   .../images/...  → .../labels/...
    Затем меняет расширение на .txt
    """
    parts = img_path.split(os.sep)
    # найдем индекс сегмента, обозначающего "изображения"
    for i, p in enumerate(parts):
        # варианты названий папок с изображениями:
        if p.startswith("image") or p == "slices":
            # сформируем соответствующую папку для лейблов
            if p.startswith("images"):
                # images, images1, images2 → labels, labels1, labels2
                label_folder = "labels" + p[len("images"):]
            elif p == "slices":
                # в sliced1 папка называется прямо labels
                label_folder = "labels"
            else:
                continue

            # подменяем сегмент и собираем новый путь
            parts[i] = label_folder
            label_path = os.sep.join(parts)
            # меняем расширение на .txt
            return os.path.splitext(label_path)[0] + ".txt"

    # если ни один сегмент не подошел — падаем
    raise ValueError(f"Не удалось сопоставить путь к разметке для {img_path}")

def load_gt(label_path, W, H):
    boxes = []
    with open(label_path) as f:
        for ln in f:
            parts = ln.split()
            if len(parts) < 5:
                continue
            xc, yc, wb, hb = map(float, parts[1:5])
            x1 = (xc - wb/2) * W
            y1 = (yc - hb/2) * H
            x2 = (xc + wb/2) * W
            y2 = (yc + hb/2) * H
            boxes.append([x1, y1, x2, y2])
    return np.array(boxes)

def compute_detection_f1(pred_boxes, gt_boxes):
    """
    pred_boxes, gt_boxes: списки numpy-двумерных массивов [N_i×4] и [M_i×4]
    Возвращает среднее F1 по порогам IoU от 0.3 до 0.93 с шагом 0.07.
    """
    thresholds = np.arange(0.3, 0.93+1e-9, 0.07)
    f1_scores = []

    for t in thresholds:
        tp = fp = fn = 0
        for p, g in zip(pred_boxes, gt_boxes):
            matched_p = set()
            matched_g = set()
            if p.size and g.size:
                inter_wh = np.maximum(
                    0,
                    np.minimum(p[:, None, 2:], g[None, :, 2:]) -
                    np.maximum(p[:, None, :2], g[None, :, :2])
                )
                inter_area = inter_wh[...,0] * inter_wh[...,1]
                area_p = (p[:,2]-p[:,0]) * (p[:,3]-p[:,1])
                area_g = (g[:,2]-g[:,0]) * (g[:,3]-g[:,1])
                union = area_p[:,None] + area_g[None,:] - inter_area
                iou_mat = inter_area / (union + 1e-8)

                idxs = np.argwhere(iou_mat >= t)
                matches = [(i,j,iou_mat[i,j]) for i,j in idxs]
                for i,j,score in sorted(matches, key=lambda x: -x[2]):
                    if i not in matched_p and j not in matched_g:
                        matched_p.add(i)
                        matched_g.add(j)

            tp += len(matched_p)
            fp += p.shape[0] - len(matched_p)
            fn += g.shape[0] - len(matched_g)

        prec = tp / (tp + fp + 1e-8)
        rec  = tp / (tp + fn + 1e-8)
        f1   = 2 * prec * rec / (prec + rec + 1e-8)
        f1_scores.append(f1)

    return float(np.mean(f1_scores))


# === 4. Сбор GT и предиктов ===
# WEIGHTS_FILE = '/kaggle/input/yolo8m-3epochs-for_inference/pytorch/default/1/best (5).pt'
# model = YOLO(WEIGHTS_FILE, task='detect')

best_model_path = os.path.join(WORK_DIR, 'yolov8s-finetune', 'weights', 'best.pt')
model = YOLO(best_model_path, task='detect')

with open(VAL_LIST) as f:
    val_imgs = [p.strip() for p in f if p.strip()]

paired_imgs = []
all_gt = []
all_pr = []

for img_path in val_imgs:
    gt_path = get_label_path(img_path)
    if not os.path.exists(gt_path):
        continue  # пропускаем без разметки

    # сохранить синхронный список
    paired_imgs.append(img_path)

    # загрузка GT
    W, H = Image.open(img_path).size
    all_gt.append(load_gt(gt_path, W, H))

    # предсказания
    res = model.predict(img_path, imgsz=IMG_SIZE, conf=CONF_THRES, verbose=False)[0]
    all_pr.append(res.boxes.xyxy.cpu().numpy())

print(f"Оценка на {len(paired_imgs)} изображениях")

# === 5. Подсчёт и вывод метрик ===
mean_f1 = compute_detection_f1(all_pr, all_gt)
print(f"Mean F1 (по IoU 0.3–0.93): {mean_f1:.4f}")


# === 6. Визуализация ===
# Вариант A: случайно 4 примера
n = min(len(paired_imgs), 4)
sample_idxs = random.sample(range(len(paired_imgs)), k=n)

# Вариант B (если хотите все подряд):
# sample_idxs = list(range(len(paired_imgs)))

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
for ax, idx in zip(axes.flatten(), sample_idxs):
    img_path = paired_imgs[idx]
    im = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    ax.imshow(im)
    ax.axis('off')

    # рисуем GT
    for x1,y1,x2,y2 in all_gt[idx]:
        ax.add_patch(Rectangle((x1,y1),
                               x2-x1, y2-y1,
                               edgecolor='lime', lw=2, fill=False))
    # рисуем предсказания
    for x1,y1,x2,y2 in all_pr[idx]:
        ax.add_patch(Rectangle((x1,y1),
                               x2-x1, y2-y1,
                               edgecolor='red', lw=2, fill=False))

plt.tight_layout()
plt.show()